In [0]:
base_dir = '/Volumes/streeming/demo_db/datasource'

spark.sql("drop table if exists streeming.demo_db.word_count")

dbutils.fs.rm(f"{base_dir}/checkpointing",True)

dbutils.fs.rm(f"{base_dir}/data/text",True)

dbutils.fs.mkdirs(f"{base_dir}/checkpointing")

dbutils.fs.mkdirs(f"{base_dir}/data/text")



In [0]:
class BatchWC():
    def __init__(self,base_dir):
        self.base_dir = base_dir

    def get_raw_data(self):
        from pyspark.sql.functions import split, col, explode
        lines_df = spark.read.format("text").option("lineSep",".").load(f"{self.base_dir}/data/text")
        raw_words_df = lines_df.select("value")\
                        .withColumn("word",explode(split(col("value")," ")))\
                        .drop("value")
        return raw_words_df
    
    def get_qality_word_count(self,df):
        from pyspark.sql.functions import lower,trim,col
        quality_word_df = df.withColumn("word",lower(trim(col("word"))))\
                              .filter(col("word").isNotNull() & col("word").rlike('[a-z]'))
        return quality_word_df
    
    def get_result(self,df):
        from pyspark.sql.functions import col
        result_df = df.groupBy("word").count().orderBy(col("count").desc())
        return result_df
        

    def write_result(self,df):
        df.write.mode("append").format("delta").saveAsTable("streeming.demo_db.word_count")


    def word_count(self):
        raw_words_df = self.get_raw_data()
        quality_word_df = self.get_qality_word_count(raw_words_df)
        result_df = self.get_result(quality_word_df)
        self.write_result(result_df)
        print("Done Writing Result to DB")

        
        
